In [1]:
from kafka import KafkaConsumer
import time
from time import sleep
from json import loads
from json import dumps
import pandas as pd
import json
import pymysql

In [ ]:
def consumer_1(p1,p2,p3,p4,p5):
    try:
        connection = pymysql.connect(host='localhost',
                                     database='mydb',
                                     user='hamz',
                                     password='453565')

        table="""INSERT INTO buyandsell (Price, Buy, Sell, BelowStopLoss, Action) 
        VALUES (%s,%s,%s,%s,%s)"""
        cursor = connection.cursor()
        result = cursor.execute(table,(p1,p2,p3,p4,p5))
        connection.commit()

        print("Record Inserted Successfully.")

    except pymysql.Error as error:
        print("Failed to insert record in MySQL: {}".format(error))
    finally:
        if connection is not None:
            cursor.close()
            connection.close()
            print("MySQL connection is closed")

In [2]:
consumer = KafkaConsumer('topic1',
                        bootstrap_servers=['localhost:9092'],
                        value_deserializer=lambda x:json.loads(x.decode('utf-8')))

In [3]:
consumer = KafkaConsumer(
    'topic1',
    bootstrap_servers=['localhost:9092'],
    value_deserializer=lambda x: loads(x.decode('utf-8'))
)

In [12]:
# Set start time and time limit for consumer
start_time = time.time()
time_limit = 30  # in seconds

In [4]:
# Define price action thresholds
buy_threshold = 81.00
sell_threshold = 82.00
stop_loss = 79.00


In [13]:
# Monitor stock prices
for message in consumer:
    # Check if time limit has been reached
    elapsed_time = time.time() - start_time
    if elapsed_time >= time_limit:
        break
    stock_data = message.value
    price = stock_data['Adj Close:CL=F']
    
    if price < buy_threshold:
        # Trigger a buy order if the price is below the buy threshold
        print(f'Price below buy threshold (${price}), triggering buy order...')
        consumer_1(price,'Yes','NO','NO','NO')
        # Place code here to trigger a buy order using your trading API
    elif price > sell_threshold:
        # Trigger a sell order if the price is above the sell threshold
        print(f'Price above sell threshold (${price}), triggering sell order...')
        consumer_1(price,'NO','YES','NO','NO')
        # Place code here to trigger a sell order using your trading API
    elif price < stop_loss:
        # Trigger a stop loss order if the price falls below the stop loss threshold
        print(f'Price below stop loss threshold (${price}), triggering stop loss order...')
        consumer_1(price,'NO','NO','YES','NO')
        # Place code here to trigger a stop loss order using your trading API    
    else:
        print(f'Price within acceptable range (${price}), no action taken.')
        consumer_1(price,'NO','NO','NO','YES')
        
    # Wait for 10 seconds before processing the next message
    time.sleep(1)

Price below buy threshold ($80.37999725341797), triggering buy order...
Price within acceptable range ($81.33999633789062), no action taken.
Price within acceptable range ($81.41999816894531), no action taken.
Price within acceptable range ($81.33000183105469), no action taken.
Price within acceptable range ($81.52999877929688), no action taken.
Price within acceptable range ($81.43000030517578), no action taken.
Price within acceptable range ($81.38999938964844), no action taken.
Price within acceptable range ($81.47000122070312), no action taken.
Price within acceptable range ($81.4800033569336), no action taken.
Price within acceptable range ($81.6500015258789), no action taken.
Price within acceptable range ($81.54000091552734), no action taken.
Price within acceptable range ($81.5199966430664), no action taken.
Price within acceptable range ($81.55000305175781), no action taken.
Price within acceptable range ($81.58999633789062), no action taken.
Price within acceptable range ($81